# Classifying Population Dynamics

In [1]:
%run ../src/dataset.py
%run ../src/augmentation.py
%run ../src/simulation.py
%run ../src/nets.py
%run ../src/trainer.py

In [2]:
ls -lah ../data

total 1.8G
drwxrwxr-x 2 folgertk folgertk 4.0K Mar 30 19:03 ./
drwxrwxr-x 6 folgertk folgertk 4.0K Mar 29 19:39 ../
-rw-rw-r-- 1 folgertk folgertk  77M Mar 29 18:18 20210329181800.ids.npy
-rw-rw-r-- 1 folgertk folgertk  120 Mar 29 18:18 20210329181800.params.json
-rw-rw-r-- 1 folgertk folgertk 268M Mar 29 18:18 20210329181800.samples.npy
-rw-rw-r-- 1 folgertk folgertk 115M Mar 29 18:18 20210329181800.theta.npy
-rw-rw-r-- 1 folgertk folgertk  77M Mar 29 19:55 20210329195537.ids.npy
-rw-rw-r-- 1 folgertk folgertk  120 Mar 29 19:55 20210329195537.params.json
-rw-rw-r-- 1 folgertk folgertk 268M Mar 29 19:55 20210329195537.samples.npy
-rw-rw-r-- 1 folgertk folgertk 115M Mar 29 19:55 20210329195537.theta.npy
-rw-rw-r-- 1 folgertk folgertk  77M Mar 29 20:22 20210329202205.ids.npy
-rw-rw-r-- 1 folgertk folgertk  120 Mar 29 20:22 20210329202205.params.json
-rw-rw-r-- 1 folgertk folgertk 268M Mar 29 20:22 20210329202205.samples.npy
-rw-rw-r-- 1 folgertk folgertk 115M Mar 29 20:22 20210329202205.

In [3]:
data = PresimulatedDataset.load("../data/20210330190331")

In [12]:
classifier = TimeseriesClassifier(MultiLayerPerceptron(128), ResNet(1))

In [13]:
train(data=data, classifier=classifier, n_epochs=100, batch_size=500, device="cuda", learning_rate=0.001)

Epoch  1: train loss = 0.52, val loss = 0.51, AUC = 0.83 ++
Epoch  2: train loss = 0.28, val loss = 0.39, AUC = 0.93 ++
Epoch  3: train loss = 0.20, val loss = 0.20, AUC = 0.97 ++
Epoch  4: train loss = 0.18, val loss = 0.17, AUC = 0.98 ++
Epoch  5: train loss = 0.17, val loss = 0.19, AUC = 0.98 --
Epoch  6: train loss = 0.16, val loss = 0.19, AUC = 0.98 --
Epoch  7: train loss = 0.16, val loss = 0.19, AUC = 0.98 --
Epoch  8: train loss = 0.17, val loss = 0.17, AUC = 0.98 --
Epoch  9: train loss = 0.15, val loss = 0.33, AUC = 0.95 --
Epoch 10: train loss = 0.16, val loss = 0.20, AUC = 0.98 --
Epoch    10: reducing learning rate of group 0 to 1.0000e-04.
Epoch 11: train loss = 0.15, val loss = 0.13, AUC = 0.99 ++
Epoch 12: train loss = 0.13, val loss = 0.13, AUC = 0.99 ++
Epoch 13: train loss = 0.12, val loss = 0.13, AUC = 0.99 ++
Epoch 14: train loss = 0.12, val loss = 0.12, AUC = 0.99 ++
Epoch 15: train loss = 0.12, val loss = 0.12, AUC = 0.99 ++
Epoch 16: train loss = 0.12, val loss 

TimeseriesClassifier(
  (embedder): ResNet(
    (resblock1): ResBlock(
      (convblock1): ConvBlock(
        (0): Conv1d(1, 64, kernel_size=(7,), stride=(1,), padding=(3,), bias=False)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (convblock2): ConvBlock(
        (0): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (convblock3): ConvBlock(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): ConvBlock(
        (0): Conv1d(1, 64, kernel_size=(1,), stride=(1,), bias=False)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (add): Add
      (act): ReLU()
  